# Automating Marine Celestial Navigation with Deep Learning

## Mount the drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# gttmw251@gmail.com
# StarNN251!

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Load useful packages

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split

## Define functions to load the data

In [3]:
def load_image(image_path, dim=(224,224)):
  """
  Loads a single image as a Numpy array and resizes it as
  desired.  The default dimensions are consistent with
  those expected by the VGG models.  

  Args:
    image_path: str pointing to the file

    dim: Two-element tuple giving the desired height
         and width of the processed image

  Returns:
    image:  A single-channel Numpy array
  """

  image = cv2.imread(image_path, 0)
  image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
  return image


def build_input(image_dir):
  """
  Loads all of the images into a single numpy array.
  Assumes that there are 101 equally-spaced images
  spanning lattitudes from 35N to 45N.  

  Args:
    image_dir: str giving name of the image directory

  Returns:
    X:  A 3-dimensional numpy array containing the
        images. Image height and width are set by
        `load_images` and default to 224 x 224.
    
    y:  A 1-dimensional numpy array of target lattitudes.
  """
  X = []
  file_paths = os.listdir(image_dir)
  for path in file_paths:
    image = load_image(os.path.join(image_dir, path))
    X.append(image.flatten())
  return np.array(X) / 255


def parse_path(image_path):
  """
  Parses a file name, extracting lattitude, longitude,
  and date/time.

  Args:
    image_path: str giving name of a path to an image
                file

  Returns:
    A tuple containing lattitude, longitude and
    date/time.
  """
  # Cut '.png'
  image_path = image_path[:-4]

  # Split on '+'
  lat, long, dtg = image_path.split('+')

  lat = float(lat)
  long = float(long)

  return (lat, long, dtg)


def parse_dir(image_dir):
  """
  Reads a file name in a directory, parses them,
  and places the data in numpy arrays.

  Args:
    image_path: str giving name of a path containing
                image files

  Returns:
    lat:  a numpy array of lattitudes
    long: a numpy array of longitudes
    dtg:  a numpy array with elements of class
          np.datetime64
  """
  path_list  = os.listdir(image_dir)
  tuple_list = [parse_path(i) for i in path_list]
  lat        = np.array([j[0] for j in tuple_list])
  long       = np.array([k[1] for k in tuple_list])
  dtg        = np.array([np.datetime64(l[2]) for l in tuple_list])
  return lat, long, dtg

In [5]:
data_path = '/content/drive/My Drive/images'
lat, long, dtg = parse_dir(data_path)
X = build_input(data_path)

In [ ]:
X.shape

(134, 50176)

In [11]:
dtg[0]

numpy.datetime64('2020-05-25T00:59:02')

## Define a custom loss function

In [ ]:
def haversine_loss(y_true, y_pred):
  # TODO: Remember that y_true and y_pred will be
  # in the from of normalized lat and long...you will
  # need to convert this to lat and long in radians.
  # The following code assumes that this is done.
  half_delta_lat  = (y_true[0] - y_pred[0]) / 2
  half_delta_long = (y_true[1] - y_pred[1]) / 2
  a = tf.sin(half_delta_lat) ** 2 + tf.cos(y_true[0]) * tf.cos(y_true[1]) + tf.sin(half_delta_long) ** 2
  c = 2 * tf.atan2(tf.sqrt(a), tf.sqrt(1 - a))
  R = 3440.065 # nautical miles
  d = R * c
  squared_d = tf.square(d)
  return tf.reduce_mean(squared_d, axis=-1)